In [1]:
from transformers import BertTokenizer, BertModel, BertForMaskedLM

In [2]:
from util.util import *

In [3]:
import torch

In [4]:
from tqdm import tqdm

In [5]:
filepath = '/data/medg/misc/phuongpm/clicr_squad_v2_mask/'

In [6]:
train = 'train1.0.json'
dev = 'dev1.0.json'
test = 'test1.0.json'

In [7]:
whs = ['what','when', 'why', 'how', 'whose', 'whom', 'where', 'which', 'who']

In [8]:
modelpath = '/crimea/phuongpm/tuned/squadv2_biobert_mask'

In [9]:
tokenizer = BertTokenizer.from_pretrained(modelpath)

In [10]:
lm =BertForMaskedLM.from_pretrained(modelpath)
lm.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [11]:
wh_ids = tokenizer.convert_tokens_to_ids(whs)

In [12]:
wh_ids

[1184, 1165, 1725, 1293, 2133, 2292, 1187, 1134, 1150]

In [13]:
def fillwhword(file, fillnone=False, remove_question_mark=False):
    rawdata = load_json(filepath + file)
    data = rawdata['data']
    for sampledata in tqdm(data):
        for p in sampledata['paragraphs']:
            for qa in p['qas']:
                oldq = qa['question']
                if fillnone:
                    newq = oldq.replace('[MASK]','')
                else:
                    tokenized_text = tokenizer.tokenize(oldq)
                    masked_index = tokenized_text.index('[MASK]')
                    segments_ids = [0] * len(tokenized_text)
                    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
                    tokens_tensor = torch.tensor([indexed_tokens])
                    segments_tensors = torch.tensor([segments_ids])
                    predictions = lm(tokens_tensor, segments_tensors)[0]
                    predictions_candidates = predictions[0, masked_index, wh_ids]
                    answer_idx = torch.argmax(predictions_candidates).item()
                    newq = oldq.replace('[MASK]', whs[answer_idx])
                if remove_question_mark:
                    newq = newq.replace('?','')
                qa['question'] = newq.strip()
    rawdata['data'] = data
    return rawdata    

In [14]:
traindata = fillwhword(train, True, True)

100%|██████████| 10638/10638 [00:00<00:00, 71788.87it/s]


In [15]:
traindata['data'][0]

{'title': 'isolated cranial distortion mimicking caput succedenum from amniotic band disruption without any neurological abnormality',
 'paragraphs': [{'context': 'summary  this report describes a term newborn with isolated distortion in the left parietal bone without any other visible congenital anomaly , due to amniotic band disruption . a skull x-ray , ultrasound scan and subsequent mri scan of the brain did not show any apparent distortion apart from depression and concavity in the left parietal bone . the purpose of this case report is to raise awareness of this possible , mild outcome of this little - known entity , which may mimic caput succedaneum ( moulding of the presenting part in the birth canal during natural delivery ) , and to provide a historical and embryological background .  background  amniotic band disruption syndrome is a rare entity which occurs in 1 in 1200 to 1 in 15 000 live births . 1 it may cause a myriad of deformities of fetal body parts from mild defects 

In [16]:
save_json(traindata, '/data/medg/misc/phuongpm/clicr_squad_v2_removeall/' + train)

In [17]:
devdata = fillwhword(dev, True, True)

100%|██████████| 584/584 [00:00<00:00, 57594.02it/s]


In [18]:
save_json(devdata, '/data/medg/misc/phuongpm/clicr_squad_v2_removeall/' + dev)

In [19]:
testdata = fillwhword(test, True, True)

100%|██████████| 624/624 [00:00<00:00, 83145.23it/s]


In [20]:
save_json(testdata, '/data/medg/misc/phuongpm/clicr_squad_v2_removeall/' + test)